# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** Data Preparation

Firstly, we import several necessary packages and load in our data:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

In [2]:
# Initializing test/train split
np.random.seed(3)
data_org = pd.read_csv(r'C:\Users\jonas\Downloads\final_data.csv')
test_data = data_org.sample(frac=0.1)
train_data = data_org.drop(test_data.index)

### **2** |  Initializations

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [3]:
# DataLoader
param_dataLoader_train = {
    "batch_size_train": 32,
    "shuffle_batch_train": True
    }

param_dataLoader_valid = {
    "batch_size_valid": 32,
    "shuffle_batch_valid": True
    }

# KFold
param_KFold = {
    "n_splits": 5,
    "shuffle": True
    }

# Model Parameter
param_model = {
    #'optimizer': 'adam'
    #'loss':
    #'metrics':
}

In [4]:
# Splitting for 5fold

kf = KFold(**param_KFold)
folds = kf.split(train_data)

In [6]:
# training
from CNN import CNN

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

for i, (train_split, valid_split) in enumerate(folds):
    model = CNN(train_data=train_data.iloc[train_split],
                valid_data=train_data.iloc[valid_split],
                architecure='dcmcdmcmfe', 
                dropouts=[.2, .2], 
                pooling=[2, 2, 2], 
                conv=[{'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'input_shape': (train_data['seq'].apply(lambda x: len(x)).max(), 4)},
                      {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'},
                      {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'}],
                dense=[{'units': 9, 'activation': 'softmax'}],
                **param_model, **param_dataLoader_train, **param_dataLoader_valid)     #initialize the model

    model.fit()
    results = model.evaluate()
    results = dict(zip(model.metrics_names, results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])

  5/311 [..............................] - ETA: 19:58 - loss: 5.4987 - accuracy: 0.1187